In [23]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from textblob import TextBlob
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer 
from nltk import word_tokenize
from langdetect import detect_langs
from wordcloud import WordCloud, STOPWORDS
import re
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm

In [2]:
appStore = pd.read_csv('AppStoreData.csv')
googlePlay = pd.read_csv('PlayStoreData.csv')

We can seperate the reviews by ratings and proceed to analysis the text in each rating category
- reduces runtime or smt
#df.loc[df['column_name'] == some_value]
#can create the function to do so

In [3]:
#filter by score and get review col
asResponse =appStore.loc[appStore['rating'] == 5].review
gpResponse = googlePlay.loc[googlePlay['score'] == 5].text


In [4]:
print(asResponse)
type(asResponse)

1      A bank like no other, no bank have such amazin...
4      I have to say that the UI/UX is one of the bes...
6      Great app, with awesome design and theme unlik...
8      Very easy to use, user friendly and intuitive....
10     Honestly is my first time depositing digital b...
                             ...                        
118                                         Love the app
121                                                 Good
122                                                = x o
125                                                  Wow
126                                                🚀🚀🚀🚀🚀
Name: review, Length: 76, dtype: object


pandas.core.series.Series

In [5]:
#asResponse = appStore.review
#gpResponse = googlePlay.reviews

In [6]:
asResponse = asResponse.values.tolist()
gpResponse = gpResponse.values.tolist()

In [7]:
#wordtokenization
wt_as = [word_tokenize(review) for review in asResponse]
wt_gp = [word_tokenize(review) for review in gpResponse]

In [8]:
print(gpResponse)

['Quick transfers to bank acc and fast repayments. Very suitable for short term loans', 'Ok Good system.', '🥰🥰🥰', "Been using this bank for a while now and didn't face any major problems. What I enjoyed the most is the saving pockets area where we could create different pockets for each saving we want to do and even add goals into it. The 2.68% per annum is added daily (0.01) as long as you have a minimum of 150 in your savings. If not, you'll have to wait till your it accumulates up to 0.01 to be added. There was also cashback when you make payment using the card which I didn't know till recently.", '👍👍👍👍', "I don't set. A password in my phone n not allow me to down load this apps", 'Very good app', 'Love GxS bank. Would it be available on Google Pay as well? 😉', 'Easy and fast app', 'Easy loan and easy payment with no hassle.', 'Very good digital savings bank account that helps grow our funds.', 'Great savings rate!', 'Loving it so far.', 'Easy to use', 'Great interest rates and card

TFIDF
As tfidf filter for words that are important in each comment, it will be essential to use it to predict the sentiment of the review is using 

In [9]:
#after word tokenizing each sentance, we can lemma/stem
#ok according to reddit, stop word removal can be harmful if we use nnm like bert or RNNS. So i wont do it for now
#we should use our own stopwords, and to find out which are the stop words, word cloud might come into play
#use tldf?
#fk it ill roll first 

#stop_words = ENGLISH_STOP_WORDS

vect = TfidfVectorizer().fit(asResponse)
vect_tfidf = vect.transform(asResponse)
print(vect_tfidf.toarray())

[[0.         0.         0.         ... 0.         0.         0.24454209]
 [0.         0.         0.         ... 0.         0.10382606 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [10]:
#General observation for the entire dataset
#calculate sentiment of sentence
sentiment_scoring = [TextBlob(review) for review in gpResponse] # this is for rating = 5
for val in sentiment_scoring:
    print(val)
    print(val.sentiment.polarity)

Quick transfers to bank acc and fast repayments. Very suitable for short term loans
0.3120833333333334
Ok Good system.
0.6
🥰🥰🥰
0.0
Been using this bank for a while now and didn't face any major problems. What I enjoyed the most is the saving pockets area where we could create different pockets for each saving we want to do and even add goals into it. The 2.68% per annum is added daily (0.01) as long as you have a minimum of 150 in your savings. If not, you'll have to wait till your it accumulates up to 0.01 to be added. There was also cashback when you make payment using the card which I didn't know till recently.
0.14464285714285713
👍👍👍👍
0.0
I don't set. A password in my phone n not allow me to down load this apps
-0.15555555555555556
Very good app
0.9099999999999999
Love GxS bank. Would it be available on Google Pay as well? 😉
0.45
Easy and fast app
0.31666666666666665
Easy loan and easy payment with no hassle.
0.43333333333333335
Very good digital savings bank account that helps gro

Roberta

In [13]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

pytorch_model.bin: 100%|██████████| 499M/499M [00:22<00:00, 22.1MB/s] 


In [16]:
example = "This oatmeal is questionable. It is mushy, soft and just so darn weird. I prefer Quaker oats"
TextBlob(example).sentiment


Sentiment(polarity=-0.3, subjectivity=0.7833333333333333)

In [21]:
#run for roberta model
encoded_text = tokenizer(example, return_tensors='pt')
output = model(**encoded_text)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
score_dict = {
    'roberta_neg' : scores[0],
    'roberta_neu' : scores[1],
    'roberta_pos' : scores[2]
}
print(score_dict)
#returns negative, neutral and positive


{'roberta_neg': 0.93354213, 'roberta_neu': 0.060158957, 'roberta_pos': 0.0062989215}


In [33]:
#do it for reviews
def roberta_score(example):
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    score_dict = {
        'roberta_neg' : scores[0],
        'roberta_neu' : scores[1],
        'roberta_pos' : scores[2]
    }
    return score_dict

In [39]:
#asResponse , gpResponse
res = {}
for i, row in tqdm(googlePlay.iterrows(), total = len(asResponse)):
    text = row['text']
    myid = row["text"]
    res[myid] = roberta_score(text)

288it [00:14, 20.03it/s]                       


In [40]:
#convert to df
result_df = pd.DataFrame(res).T

In [41]:
print(result_df.head())

                                                    roberta_neg  roberta_neu  \
cant logging using finger print . previously st...     0.879858     0.107318   
For 1st time registration signup by singpass an...     0.856186     0.137936   
Updated - Kindly ignore my previous rating, and...     0.460258     0.496379   
They unable to process my application. 4x attempt      0.893826     0.099285   
Try logging in after logging out completely. Ca...     0.799691     0.186849   

                                                    roberta_pos  
cant logging using finger print . previously st...     0.012824  
For 1st time registration signup by singpass an...     0.005878  
Updated - Kindly ignore my previous rating, and...     0.043363  
They unable to process my application. 4x attempt      0.006889  
Try logging in after logging out completely. Ca...     0.013459  


using transformer pipeline

In [42]:
from transformers import pipeline
sent_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
config.json: 100%|██████████| 629/629 [00:00<00:00, 2.04MB/s]
model.safetensors: 100%|██████████| 268M/268M [00:08<00:00, 30.0MB/s] 
tokenizer_config.json: 100%|██████████| 48.0/48.0 [00:00<00:00, 25.7kB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 9.28MB/s]


In [43]:
sent_pipeline("this application is sooooo good that i will never ever touch it again")

[{'label': 'POSITIVE', 'score': 0.9997859597206116}]